# <b> sagemaker 실행 환경설정 </b>


#### AutoReload

이 코드는 Jupyter Notebook이나 IPython 환경에서 모듈을 자동으로 다시 로드하는 기능을 활성화합니다.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
install_needed = True  # should only be True once
# install_needed = False

SageMaker 노트북 인스턴스일 경우 docker image 저장소의 공간이 작아 마운트한 EBS로 변경을 진행합니다.
SageMaker 노트북 인스턴스에서 로컬 모드 디버깅 시 종종 No space left 관련 오류가 발생합니다. 따라서, 도커 이미지/컨테이너가 저장될 폴더를 SageMaker EBS (Amazon Elastic Block Store) 볼륨으로 변경하는 것을 권장합니다. 도커 이미지/컨테이너는 기본적으로 EBS가 아닌 루트 볼륨에 저장하기 때문에(루트 볼륨의 크기는 사용자가 임의로 조정할 수 없습니다!) 고용량의 이미지들을 빌드하면 용량이 꽉 차기 때문입니다.

In [4]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Add data-root and default-shm-size=10G


Redirecting to /bin/systemctl restart docker.service


Docker Restart


In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    #!{sys.executable} -m pip install -U smdebug sagemaker-experiments-logger sagemaker-experiments awswrangler==2.20.1
    !{sys.executable} -m pip install -U sagemaker
    #!{sys.executable} -m pip install -U datasets transformers
    !sudo curl -L "https://github.com/docker/compose/releases/download/v2.27.1/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
    !sudo chmod +x /usr/local/bin/docker-compose
        
    IPython.Application.instance().kernel.do_shutdown(True)

## AWS IAM ROLE 설정

In [4]:
# 현재 SageMaker Execution Role Name을 가져오는 코드
from sagemaker import get_execution_role
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/27/25 15:49:06] INFO     Found credentials from IAM Role:                                   ]8;id=171569;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=518114;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[05/27/25 15:49:07] INFO     Found credentials from IAM Role:                                   ]8;id=147482;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=584596;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

SageMaker Execution Role Name: SagemakerWorkshopFullAccessRole


### 1.1 SageMaker Execution Role에 IAM 정책 연결하기 (<b>콘솔 사용</b>)

> step 1. AWS 콘솔 > IAM으로 이동  
>
> step 2. 왼쪽 네비게이터에서 "Role" 선택  
> ![nn](../assets/images/Role.png)  
>
> step 3. SageMaker Execution Role을 검색 후, 해당 role 선택 (상위 셀의 출력 참고)  
> ![nn](../assets/images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택  
> ![nn](../assets/images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" 정책을 검색 후 attach  
> ![nn](../assets/images/attach-policy.png) 


In [6]:
import sys
sys.path.append("../assets")
from utils.iam import iam_handler
iam = iam_handler()
strID = "DJ"
strPrefix = f'{strID}-SM-IMD'

[05/27/25 15:55:35] INFO     Found credentials from IAM Role:                                   ]8;id=81028;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=845640;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 1.2 Attach policy - sagemaker execution role

아래 정책들을 SageMaker Execution Role에 연결합니다.  
각 서비스에 대한 전체 액세스 권한을 부여하여 SageMaker에서 다양한 AWS 리소스를 사용할 수 있도록 합니다.

> ECR  <BR> 
> S3 <BR>
> SSM <BR> 
> CodeCommit <BR>
> CodeBuild <BR>
> CodePipeline <BR>
> EventBridge <BR>
> Lambda



In [7]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeCommitFullAccess",
    #"arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess",
    "arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess",
    "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
    "arn:aws:iam::aws:policy/AWSLambda_FullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

## 2. Set default parameters

In [9]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [10]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [11]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)

#pm.put_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"]), value=lambdaRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"]), value=buildRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"]), value=pipelineRoleArn, overwrite=True)
#pm.put_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"]), value=eventBridgeRoleArn, overwrite=True)

'Store suceess'

In [12]:
print ("REGION: ", pm.get_params(key="-".join([strPrefix, "REGION"])))
print ("BUCKET: ", pm.get_params(key="-".join([strPrefix, "BUCKET"])))
print ("ACCOUNT-ID: ", pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"])))

print ("SAGEMAKER-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"])))
#print ("LAMBDA-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "LAMBDA-ROLE-ARN"])))
#print ("CODE-BUILD-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-BUILD-ROLE-ARN"])))
#print ("CODE-PIPELINE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-PIPELINE-ROLE-ARN"])))
#print ("CODE-EVENTBRIDGE-ROLE-ARN: ", pm.get_params(key="-".join([strPrefix, "CODE-EVENTBRIDGE-ROLE-ARN"])))

REGION:  us-east-1
BUCKET:  sagemaker-us-east-1-740128158828
ACCOUNT-ID:  740128158828
SAGEMAKER-ROLE-ARN:  arn:aws:iam::740128158828:role/SagemakerWorkshopFullAccessRole
